# 데이터 전처리(3)

In [1]:
# 라이브러리 선언
import pandas as pd
import numpy as np

# 데이터 불러오기
joinMaster = pd.read_csv("../../dataset/join_region_master.csv", encoding = "ms949")
joinSellout = pd.read_csv("../../dataset/join_kopo_product_volume.csv", encoding = "ms949")

In [2]:
joinMaster

,REGIONID,REGIONNAME
0,A01,한국
1,A02,미국
2,A03,프랑스
3,A04,독일


In [3]:
joinSellout

,REGIONID,PRODUCT,YEARWEEK,QTY
0,A01,ST0001,201901,3000
1,A01,ST0001,201902,3500
2,A02,ST0002,201901,4000
3,A02,ST0003,201902,7000


In [5]:
# 1. 데이터 병합 (1)

# pd.merge(left = dfname1, right = dfname2,
#        left_on = dfname1_joinkey, right_on = dfname2_joinkey, how = joinoptions)

# joinopt = inner, left, right, left
# inner : 공통 필드를 갖는 경우에 대해서만 join, NaN값이 나오지 않는다.
# outer : inner + 한가지만 충족하는 경우에도 join한다. NaN값이 출력될 수 있다.
# left : left_on의 df기준 키값을 충족하는 경우에만 적용. NaN값이 출력될 수 있다.
# right : right_on의 df기준 키값을 충족하는 경우에만 적용. NaN값이 출력될 수 있다.

# joinkey는 중복을 허락하지 않는 기본키여야 한다.

# 기본키가 중복되지 않을 때 - 중복없이 값이 할당된다.
pd.merge(left = joinSellout, right = joinMaster,
        on = "REGIONID", how = "inner")

,REGIONID,PRODUCT,YEARWEEK,QTY,REGIONNAME
0,A01,ST0001,201901,3000,한국
1,A01,ST0001,201902,3500,한국
2,A02,ST0002,201901,4000,미국
3,A02,ST0003,201902,7000,미국


In [8]:
testMaster = joinMaster.loc[0:1]

In [9]:
testMaster.loc[testMaster.REGIONID=="A02", "REGIONID"] = "A01"

In [10]:
testMaster

,REGIONID,REGIONNAME
0,A01,한국
1,A01,미국


In [11]:
# 기본키가 중복될 때 - 적용대상이 모호하여 데이터가 2배로 늘어난다.
pd.merge(left = joinSellout, right = testMaster,
        on = "REGIONID", how = "inner")

,REGIONID,PRODUCT,YEARWEEK,QTY,REGIONNAME
0,A01,ST0001,201901,3000,한국
1,A01,ST0001,201901,3000,미국
2,A01,ST0001,201902,3500,한국
3,A01,ST0001,201902,3500,미국


In [12]:
join2 = joinMaster.rename(columns={"REGIONID":"AP2ID"})

In [15]:
join2

,AP2ID,REGIONNAME
0,A01,한국
1,A01,미국
2,A03,프랑스
3,A04,독일


In [16]:
join2.loc[4, "AP2ID"] = "A05"
join2.loc[4, "REGIONNAME"] = "인도"

In [17]:
join2

,AP2ID,REGIONNAME
0,A01,한국
1,A01,미국
2,A03,프랑스
3,A04,독일
4,A05,인도


In [14]:
pd.merge(left = joinSellout, right = join2,
        left_on = "REGIONID", right_on="AP2ID", how = "inner")

,REGIONID,PRODUCT,YEARWEEK,QTY,AP2ID,REGIONNAME
0,A01,ST0001,201901,3000,A01,한국
1,A01,ST0001,201901,3000,A01,미국
2,A01,ST0001,201902,3500,A01,한국
3,A01,ST0001,201902,3500,A01,미국


In [23]:
# 2. 데이터 병합 (2)

# 샘플데이터
colList = ["REGIONID", "PRODUCT", "DATE", "QTY"]

data1 = pd.DataFrame([["서울", "1 1 카드", 20210518, 100],
["서울", "1 1 카드", 20210517, 120],
["서울", "1 1 카드", 20210516, 103],
["서울", "1 1 카드", 20210515, 104],
["서울", "1 1 카드", 20210514, 150]], columns = colList)

data2 = data1.copy()
data3 = data1.copy()
data4 = data1.copy()

data2.PRODUCT = "2 2 카드"
data3.REGIONID = "부산"
data4.PRODUCT = "3 3 카드"

mergedData = data1.append(data2).append(data3).append(data4)
promotionCol = ["SUBSIDIARY", "CARD_NAME", "PROMOTION_NAME", "PROMOTION_RATE"]
promotionData = pd.DataFrame([
    ["서울", "1 1 카드", "서울_프로모션", 0.5],
    ["부산", "2 2 카드", "부산_프로모션", 0.3],
    ["대구", "3 3 카드", "대구_프로모션", 0.9],
    ["광주", "1 1 카드", "광주_프로모션", 0.7]], columns = promotionCol)

In [87]:
mergedData.merge(promotionData, left_on = "PRODUCT", right_on = "CARD_NAME", how = "left")

,REGIONID,PRODUCT,DATE,QTY,SUBSIDIARY,CARD_NAME,PROMOTION_NAME,PROMOTION_RATE
0,서울,1 1 카드,20210518,100,서울,1 1 카드,서울_프로모션,0.5
1,서울,1 1 카드,20210518,100,광주,1 1 카드,광주_프로모션,0.7
2,서울,1 1 카드,20210517,120,서울,1 1 카드,서울_프로모션,0.5
3,서울,1 1 카드,20210517,120,광주,1 1 카드,광주_프로모션,0.7
4,서울,1 1 카드,20210516,103,서울,1 1 카드,서울_프로모션,0.5
5,서울,1 1 카드,20210516,103,광주,1 1 카드,광주_프로모션,0.7
6,서울,1 1 카드,20210515,104,서울,1 1 카드,서울_프로모션,0.5
7,서울,1 1 카드,20210515,104,광주,1 1 카드,광주_프로모션,0.7
8,서울,1 1 카드,20210514,150,서울,1 1 카드,서울_프로모션,0.5
9,서울,1 1 카드,20210514,150,광주,1 1 카드,광주_프로모션,0.7


In [91]:
# 컬럼에 대한 다중조건 부여, 실적 위주의 데이터 열람
pd.merge(left = mergedData, right = promotionData,
        left_on = ["REGIONID", "PRODUCT"], right_on = ["SUBSIDIARY", "CARD_NAME"], how = "left")

,REGIONID,PRODUCT,DATE,QTY,SUBSIDIARY,CARD_NAME,PROMOTION_NAME,PROMOTION_RATE
0,서울,1 1 카드,20210518,100,서울,1 1 카드,서울_프로모션,0.5
1,서울,1 1 카드,20210517,120,서울,1 1 카드,서울_프로모션,0.5
2,서울,1 1 카드,20210516,103,서울,1 1 카드,서울_프로모션,0.5
3,서울,1 1 카드,20210515,104,서울,1 1 카드,서울_프로모션,0.5
4,서울,1 1 카드,20210514,150,서울,1 1 카드,서울_프로모션,0.5
5,서울,2 2 카드,20210518,100,NaN,NaN,NaN,NaN
6,서울,2 2 카드,20210517,120,NaN,NaN,NaN,NaN
7,서울,2 2 카드,20210516,103,NaN,NaN,NaN,NaN
8,서울,2 2 카드,20210515,104,NaN,NaN,NaN,NaN
9,서울,2 2 카드,20210514,150,NaN,NaN,NaN,NaN
